This notebook focuses on examples inspired by Kurose & Ross's Computer Networking: A Top-Down Approach.

As always, we start by loading ns-3.

In [2]:
import sys
sys.path.append("./ns-3-dev/build/bindings/python")
sys.path.append("./ns-3-dev/build/lib")
from ns import ns

[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { _GLOBAL__sub_I_cling_module_136, $.cling-module-136.__inits.0, __orc_init_func.cling-module-136, _ZN3ns3L16g_timeInitHelperE, __cxx_global_var_initcling_module_136_ }) }
[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { __orc_init_func.cling-module-136 }) }


Now we can start building our examples. Remember to always destroy the simulator state before rerunning cells.

In [2]:
ns.Simulator.Destroy()

# First example: network stack, packet and flows (based on 6th Edition, Chapter 1 & 2)

Computer networks are like onions. Layers! 

Onions have layers, and computer network have layers (yes, this is a Shrek reference). 

The widely used Internet, or TCP/IP, stack is composed of five layers.

```mermaid
flowchart 

subgraph NS[Network Stacks]
  direction LR

subgraph stackA[Node 1]
  direction TB
  A[Application]-->|sockets|B[Transport];
  B-->C[Network]; 
  C-->D[Link];
  D-->E[Physical];
end
subgraph stackB[Node 2]
  direction BT
  J[Physical]-->I[Link];
  I-->H[Network];
  H-->G[Transport]; 
  G-->|sockets|F[Application];
end
stackA-->|physical medium|stackB;
end
```

The application layer is where applications such as web browsers, games, office suites, banking  services, etc, generate and consume payloads that will be transported by the network.

The transport layer acts as an intermediate between the application and the underlying network, implementing basic services/protocols or more complex ones, that try to offer more guarantees to the application, such as error checking and automatic retransmission, automatic ordering, etc. Packets that arrive or leave the transport layer have source and destination ports, that identify which process at the sender and the receiver are trying to communicate across a network route composed of many links.

The network layer implements logical routes. These routes are based on forwarding packets from one network node to another, connected by a physical link. These packet forwardings are also known as hops, which are limited via a counter called Time-To-Live (TTL). When TTL reaches 0, the packet is discarded, and the original sender retransmits it (in case it is using a reliable transport protocol).

The link layer and physical layer are typically combined in a network interface, or network device. But you probably know them by their popular names "network card" and "modem". The link layer is also commonly referred to as the Medium-Access Control (MAC) layer, because it controls the physical layer access to the physical media.

The physical layer of both sender (Node 1) and receiver (Node 2) are then physically connected by a physical medium, such as air (e.g. Morse code, Wi-Fi, li-fi, 2G/3G/4G/5G/6G, and even smoke signal), water (e.g. UAN), metals such as copper (e.g. telegraph, landline, Ethernet, thunderbolt, USB), glass (e.g. optical fiber), plastic (e.g. TOSLINK).  

The physical medium, or part of its capacity to transmit data, is typically referred to as a channel. The channel may have different capacities depending on the medium properties. Optical fiber, for example, is far less susceptible to interference, allowing for much higher capacity than copper, especially in long links, such as the submarine cables that connect countries and continents to a single global internet.

Enough with the chit-chat, let us see a real example of a network stack, built step by step.

In [4]:
from ns import ns
def simu_stack():
    ns.Simulator.Destroy()

    # First we create a channel (the physical medium) that will connect multiple devices.
    # In this case, it will be the metal bus inside a hub, connected to all the usual 4 RJ-45 ports
    hubBus = ns.CreateObject[ns.CsmaChannel]().GetObject[ns.CsmaChannel]()
    hubBus.SetAttribute("DataRate", ns.StringValue("100Mbps"))

    # Now we need devices to connect to our hub
    nodes = ns.NodeContainer()
    nodes.Create(2)

    # These devices will contain the network interfaces/cards, which are typically composed of Link/MAC and Physical/PHY layers
    for i in range(2):
        # Let's install create a new network card
        networkInterface = ns.CreateObject[ns.CsmaNetDevice]()

        # MAC address set by the manufacturer
        networkInterface.SetAddress(ns.Mac48Address.Allocate().ConvertTo())

        # Configure a packet queue (in an actual hardware, it would be a memory buffer managed by the network card controller)
        queue = ns.CreateObject[ns.DropTailQueue[ns.Packet]]()
        networkInterface.SetQueue(queue)

        # Then finally connect the network card to the motherboard
        nodes.Get(i).AddDevice(networkInterface)

        # And then connect the cable between the hub and the network card
        networkInterface.Attach(hubBus)

        # To show this is actually working, we are going to capture the network traffic through this device,
        # like you can do with Wireshark on your own computer and network
        pcapHelper = ns.PcapHelper()
        filename = pcapHelper.GetFilenameFromDevice("simu_stack", networkInterface)
        file = pcapHelper.CreateFile(filename, ns.cppyy.gbl.std.ios.out, ns.PcapHelper.DLT_EN10MB)
        pcapHelper.HookDefaultSink[ns.CsmaNetDevice](networkInterface, "Sniffer", file)

    # At this point we have physical medium, phy and mac connected, but none of the upper layers responsible for routing,
    # implementing reliable transport, multiplexing applications, and the applications themselves

    # Let's first install the TCP/IP stack for routing, reliable transport and application multiplexing
    tcpipStack = ns.InternetStackHelper()
    tcpipStack.Install(nodes)

    # We've got the "hardware" in place.  Now we need to add IP addresses.
    # In consumer equipment we tend to see 192.168.0.0/24
    ipv4h = ns.Ipv4AddressHelper()
    ipv4h.SetBase("192.168.0.0", "255.255.255.0")
    ipv4h.Assign(nodes.Get(0).GetDevice(0))
    ipv4h.Assign(nodes.Get(1).GetDevice(0))

    # Now our network and transport stacks are fully configured. We just need some application
    # Create a web server in the node 0
    httpServerHelper = ns.UdpEchoServerHelper(80)
    server = httpServerHelper.Install(nodes.Get(0))

    # Create a web client (e.g. browser) in the node 1
    httpClientHelper = ns.UdpEchoClientHelper(ns.InetSocketAddress(ns.Ipv4Address("192.168.0.1"), 80).ConvertTo())
    client = httpClientHelper.Install(nodes.Get(1))

    # Since we are in a simulator, we need to specify when these applications start and stop
    server.Start(ns.Seconds(1))
    client.Start(ns.Seconds(2))

    # And we need to specify when the simulation stops
    ns.Simulator.Stop(ns.Seconds(10))
    ns.Simulator.Run()

    # Let us print the packets captured in the channel by the server
    def print_packets_with_tshark(packet_i):
        import pyshark # requires `apt install tshark` + `pip install pyshark`
        cap = pyshark.FileCapture('simu_stack-0-0.pcap')
        i = 0
        for packet in cap:
            if packet_i == i:
                print(packet)
                break
            i += 1
        cap.close()

    # Jupyter shenanigans with Pyshark asyncio code
    import concurrent.futures
    def exec_async(func, *args, **kwargs):
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            future = executor.submit(func, *args, **kwargs)
        return future.result()
    exec_async(print_packets_with_tshark, packet_i=10)

simu_stack()

Packet (Length: 146)
Layer ETH
:	Destination: 00:00:00:00:00:01
	Address: 00:00:00:00:00:01
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 00:00:00:00:00:02
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Frame check sequence: 0x00000000 [unverified]
	FCS Status: Unverified
	Address: 00:00:00:00:00:02
Layer IP
:	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 128
	Identification: 0x0003 (3)
	Flags: 0x00
	0... .... = Reserved bit: Not set
	.0.. .... = Don't fragment: Not set
	..0. .... = More fragments: Not set
	...0 0000 

There we have it. We have our network composed of two nodes, with their respective stacks, connected by a channel.

From the text printed, we can check the contents of the 10th packet transmitted/received by the web server.

Note that the packet contains multiple sections: Layer ETH, Layer IP, Layer UDP, DATA. These correspond to the link (Ethernet MAC), network (IPv4), transport (UDP) and application layers (user generated payload).

Like previously said, the payload starts at the application layer, where it receives additional data to perform the subsequent layer duties. The most relevant data that gets prepended to the application payload by the transport layer are the source and destination ports, which identify the application process running on the peer that is generating a message and its destination process on the other peer.

In the next layer, the most important data prepended to the transport layer segment is the destination and source addresses, in this case, the IP addresses. These addresses allow for a packet to be forwarded via a series of interlinked small networks that logically compose the internet, or a private WAN. The packet at the network layer is called datagram.

In the MAC layer, the datagram receives additional MAC addresses, being called a frame. The MAC addresses allow devices sharing the same physical medium to send/receive messages between them with no intermediates, a process known as forwarding.

The network layer implements routing as a series of forwardings, where the forwarding intermediates receive a packet MAC addressed to them, then check the destination IP address, and in case it isn't them, forward to a different network port or discards it, according to the rules in its own routing table.

A communication pair, composed of source and destination IP addresses and ports, is known as a network flow. When two devices in the network communicate between themselves over the same IPs and ports, we have two flows, one for uplink and the other for downlink (in reference to one of the peers).

# Second example: network delay (based on 6th Edition, Chapter 1, R18 and P7)


# Third example: network bottlenecks (based on 6th Edition, Chapter 1, R19)

Computer networks are formed by a series of links connecting devices physically via a medium.

Those links can be interconnected logically via routes, in which nodes receive packets not meant for them, and forward them to other links, until the packet reaches its final destination or is discarded.

You may have experienced a side effect of those logical routes, for example when you subscribe to a 1Gbps internet plan, but has to wait for downloads to complete, because downloads are still at meager 10s-100s of Mbps.

This typically happen due to bottlenecks in the route, which are limited by the lowest capacity link. This is why Speedtest.net and other similar services typically pick a nearby server to measure your internet speed, reducing the number of link in the route, which translates to a smaller chance of being bottlenecked by a link other than your ISP network and the remote server.

Let's see this bottleneck in action by building a network composed of four nodes, connected by links of different capacities.

```mermaid
flowchart LR
A[A];
B[B];
C[C];
D[D];
A-->|500kbps|B;
B-->|2Mbps|C; 
C-->|1Mbps|D;
```

We want to send a big payload from A to D, for example 4MB. We also want to find out how long it takes to send this file.
The simulation for this can be modelled as follows:

In [3]:
def simu_bottleneck(link_thrs=["500kbps", "2Mbps", "1Mbps"]):
    ns.Simulator.Destroy()
    topology = ns.NodeContainer(4) # create the nodes A, B, C, D
    for i in range(3):
        # Create a node container for each pair
        cn = ns.NodeContainer()
        cn.Add(topology.Get(i))
        cn.Add(topology.Get(i+1))
               
        # Install a point-to-point link between them
        csma = ns.CsmaHelper()
        csma.SetChannelAttribute("DataRate", ns.StringValue(link_thrs[i]))
        devices = csma.Install(cn)

        # Install the IP stack
        stack = ns.InternetStackHelper()
        stack.Install(cn)

        # Set Ipv4 Addresses
        address = ns.Ipv4AddressHelper()
        address.SetBase(ns.Ipv4Address(f"10.1.{i}.0"), ns.Ipv4Mask("255.255.255.0"))
        interfaces = address.Assign(devices)

    # Populate routing tables
    ns.Ipv4GlobalRoutingHelper.PopulateRoutingTables()

    # Now that we have our topology, we need to setup our application to inject 
    # 4MB of traffic from A to D, and a sink application at D to receive this data
    sourceApplications = ns.ApplicationContainer()
    sinkApplications = ns.ApplicationContainer()

    # Install traffic source at node A
    ipv4 = topology.Get(3).GetObject[ns.Ipv4]()
    address = ipv4.GetAddress(1, 0).GetLocal()
    sourceHelper = ns.OnOffHelper("ns3::UdpSocketFactory", ns.InetSocketAddress(address, 80).ConvertTo())
    # the application should transmit with a high data rate to saturate the links
    sourceHelper.SetAttribute("DataRate", ns.DataRateValue(ns.DataRate("10Mbps"))) 
    sourceHelper.SetAttribute("MaxBytes", ns.UintegerValue(4*1024*1024))  # transmit a total of 4 MB
    sourceHelper.SetAttribute("PacketSize", ns.UintegerValue(1024))  # in packets of 1 KB
    sourceHelper.SetAttribute("OffTime", ns.StringValue("ns3::ConstantRandomVariable[Constant=0.0]")) # always transmit
    sourceApplications.Add(sourceHelper.Install(topology.Get(0)))
    sourceApplications.Start(ns.Seconds(1.0))

    # Create sink sink at node D
    packetSinkHelper = ns.PacketSinkHelper("ns3::UdpSocketFactory", ns.InetSocketAddress(ns.Ipv4Address.GetAny(), 80).ConvertTo())
    sinkApplications.Add(packetSinkHelper.Install(topology.Get(3)))
    sinkApplications.Start(ns.Seconds(1))

    flowmon_helper = ns.FlowMonitorHelper()
    monitor = flowmon_helper.InstallAll()

    # Start simulator
    ns.Simulator.Stop(ns.Minutes(1))
    ns.Simulator.Run()

    # Print results
    print("flows", len(list(monitor.GetFlowStats())))
    flow_id, flow_stats = list(monitor.GetFlowStats())[0]
    txBytes = flow_stats.txBytes
    rxBytes = flow_stats.rxBytes
    txTime = (flow_stats.timeLastRxPacket-flow_stats.timeFirstRxPacket).GetSeconds()
    thr = rxBytes*8/txTime
    thrUnits = ["bps", "kbps", "Mbps", "Gbps", "Tbps"]
    while thr > 1024:
        thr /= 1024
        thrUnits.pop(0)
    print("Total bytes sent:", txBytes)
    print("Total bytes received:", rxBytes)
    print("Total transmission time:", txTime)
    print("Throughput:", thr, thrUnits[0])

simu_bottleneck()

flows 1
Total bytes sent: 4308992
Total bytes received: 810040
Total transmission time: 13.303388
Throughput: 475.7011898021767 kbps


The simulation results from above tell us that A transmitted ~4.3MB in approximately 13 seconds (the 0.3 MB come from packet header and/or trailer overhead, added at each network layer).

Only 810 kilobytes were received by D, at an average throughput of 475 kbps.

Note that the observed throughput is very close to the lowest capacity link between A and B, which is what we expected.

To double-check we didn't make any mistake, we can repeat our experiment lowering the capacity of the link between B and C, for example, from 2 Mbps to 100 kbps.

```mermaid
flowchart LR
A[A];
B[B];
C[C];
D[D];
A-->|500kbps|B;
B-->|100kbps|C; 
C-->|1Mbps|D;
```

In [4]:
simu_bottleneck(["500kbps", "100kbps", "1Mbps"])

flows 1
Total bytes sent: 4308992
Total bytes received: 269312
Total transmission time: 22.071774
Throughput: 95.32536895312538 kbps


The transmission time increased, the number of received bytes decreased, and the throughput was also reduced.

Note that the throughput is close to the lowest capacity link between B and C, as predicted.

Now let us see what happens if our internet provider (e.g. A-B link) is upgraded, and the core network links are of higher capacity.

In [6]:
simu_bottleneck(["10Mbps", "1Gbps", "1Gbps"])

flows 1
Total bytes sent: 4308992
Total bytes received: 4205896
Total transmission time: 3.457587552
Throughput: 9.280586379611725 Mbps


Yet again, the achievable throughput is bound by the link with the lowest capacity.

Remember this is the case before calling to your ISP and cursing the poor attendants that do not control the network infrastructure.

# Fourth example: sockets (based on 6th Edition, Chapter 2, R19)
